<a href="https://colab.research.google.com/github/Gilgamesh60/Alexnet_CNN_CIFAR10/blob/main/Alexnet_model_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [28]:
mean=(0.49139968,0.48215827,0.44653124)
std=(0.24703233,0.24348505,0.26158768)
transformer=transforms.Compose([transforms.ToTensor(),transforms.Normalize(torch.Tensor(mean),torch.Tensor(std))])

train_dataset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transformer)

test_dataset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transformer)

train_set=torch.utils.data.DataLoader(train_dataset,shuffle=True,batch_size=128)
test_set=torch.utils.data.DataLoader(test_dataset,shuffle=True,batch_size=128)

Files already downloaded and verified
Files already downloaded and verified


In [42]:
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.conv import Conv2d
from torch.nn.modules.activation import ReLU
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 96, kernel_size= 7, stride=2, padding=2 )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc1  = nn.Linear(in_features= 256*3*3, out_features= 1024)
        self.fc2  = nn.Linear(in_features= 1024, out_features= 512)
        self.fc3 = nn.Linear(in_features=512 , out_features=10)


    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [44]:
from torch.nn.modules.container import ModuleList
from torch import optim
#training the model
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=AlexNet()
model=model.to(device=device)

learning_rate=1e-4
load_model=True
epochs=20
loss_function = nn.CrossEntropyLoss()
optimizer =optim.Adam(params= model.parameters(),lr=learning_rate)

for epoch in range(epochs):
    
    for image, label in train_set:
        image = image.to(device)
        label = label.to(device)

        output = model(image)

        loss = loss_function(output,label)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
    
    print(f'Epoch {epoch+1}/{epochs}: Training Loss: {loss.item()}')


print("Training of alexNet model successfully completed!")

Epoch 1/20: Training Loss: 1.5058481693267822
Epoch 2/20: Training Loss: 1.2421005964279175
Epoch 3/20: Training Loss: 1.0785176753997803
Epoch 4/20: Training Loss: 0.8885512351989746
Epoch 5/20: Training Loss: 0.7404189705848694
Epoch 6/20: Training Loss: 0.6424452662467957
Epoch 7/20: Training Loss: 0.6696251630783081
Epoch 8/20: Training Loss: 0.6983970999717712
Epoch 9/20: Training Loss: 0.6608685255050659
Epoch 10/20: Training Loss: 0.6649959087371826
Epoch 11/20: Training Loss: 0.4316275715827942
Epoch 12/20: Training Loss: 0.31155887246131897
Epoch 13/20: Training Loss: 0.24236683547496796
Epoch 14/20: Training Loss: 0.4039763808250427
Epoch 15/20: Training Loss: 0.2936595380306244
Epoch 16/20: Training Loss: 0.23694773018360138
Epoch 17/20: Training Loss: 0.08385704457759857
Epoch 18/20: Training Loss: 0.24220497906208038
Epoch 19/20: Training Loss: 0.18192777037620544
Epoch 20/20: Training Loss: 0.12378142774105072
Training of alexNet model successfully completed!


In [50]:
correct=0
incorrect=0

with torch.no_grad():
  for image,label in test_set:
      image = image.to(device)
      label = label.to(device)

      output = model(image)
      temp,pred=torch.max(output,1)
      correct+= (pred==label).sum().item()
      incorrect+= (pred!=label).sum().item()

      model_accuracy =(correct/(correct+incorrect))*100
  
  
  
print("The model accuracy is :",model_accuracy,"%",sep=" ")


The model accuracy is : 71.87 %
